In [1]:
import re
import os
import tensorflow as tf
import numpy as np
import idx2numpy as idx2np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def save_idx(arr, fname):
    if arr.shape == ():
        arr = np.array([arr], dtype=arr.dtype)
    if arr.dtype in [np.int64]:
        print("unsupported int format for idx detected: {}, using int32 instead".format(arr.dtype))
        arr = arr.astype(np.int32)
    out_dir = os.path.dirname(fname)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir)
    with open(fname, "wb") as fid:
        idx2np.convert_to_file(fid, arr)
    print("{} saved".format(fname))

In [3]:
def save_consts(sess, out_dir="."):
    out_dir = os.path.expanduser(out_dir)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    graph = sess.graph
    graph_def = sess.graph.as_graph_def()
    for node in graph_def.node:
        if node.op == "Const":
            op = graph.get_operation_by_name(node.name)
            for out_tensor in op.outputs:
                arr = out_tensor.eval()
                tname = re.sub(r'[/:]', '_', out_tensor.name)
                idx_fname = os.path.join(out_dir, "{}.idx".format(tname))
                save_idx(arr, idx_fname)

In [4]:
def save_graph(graph, graph_name="graph", out_dir="."):
    out_dir = os.path.expanduser(out_dir)
    graph_fname = os.path.join(out_dir, "{}.pb".format(graph_name))
    with tf.gfile.FastGFile(graph_fname, "wb") as fid:
        fid.write(graph.as_graph_def().SerializeToString())
    print("{} saved".format(graph_fname))

-----

## Test Const

In [31]:
!rm -f const/*.idx

In [32]:
# test case for Const
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.arange(10), dtype=tf.float32, name="x")

In [33]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "const/")
    save_graph(graph, "test_const", "const")

const/x_0.idx saved
const/test_const.pb saved


In [34]:
!python3 quantize_graph.py --input const/test_const.pb \
  --output const/test_quant_const.pb --mode=eightbit --output_node_name='x'

-----

## Test Add

In [35]:
!rm -f add/*.idx

In [36]:
# test case for Add
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.array([1, 2, 3, 4]), dtype=tf.float32, name="x")
    y = tf.constant(np.array([1, 1, 1, 1]), dtype=tf.float32, name="y")
    z = tf.add(x, y, name="z")

In [37]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, 'add/')
    save_graph(graph, "test_add", "add")
    np_z = z.eval()
    save_idx(np_z, "add/output_z.idx")

add/x_0.idx saved
add/y_0.idx saved
add/test_add.pb saved
add/output_z.idx saved


In [38]:
!python3 quantize_graph.py --input add/test_add.pb \
  --output add/test_quant_add.pb --mode=eightbit --output_node_name='z'

-----

## Test Argmax

In [ ]:
!rm -f argmax/*.idx

In [13]:
# test case for argmax
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.random.rand(5, 3), dtype=tf.float32, name="x")
    arg_max = tf.argmax(x, axis=1, name='argmax')

In [14]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "argmax")
    save_graph(graph, "test_argmax", "argmax")
    np_argmax = arg_max.eval()
    save_idx(np_argmax, "argmax/output_argmax.idx")

argmax/x_0.idx saved
argmax/argmax_dimension_0.idx saved
argmax/test_argmax.pb saved
unsupported int format for idx detected: int64, using int32 instead
argmax/output_argmax.idx saved


In [15]:
!python3 quantize_graph.py --input argmax/test_argmax.pb \
  --output argmax/test_quant_argmax.pb --mode=eightbit --output_node_name='argmax'

-----

## Test Matmul

In [17]:
# test for matmul
graph = tf.Graph()
with graph.as_default():
    w = tf.constant(np.arange(1, 10).reshape((3, 3)), dtype=tf.float32, name="w")
    x = tf.constant(np.ones((3, 1))/3, dtype=tf.float32, name="x")
    z = tf.matmul(w, x, name='z')

In [18]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "matmul")
    save_graph(graph, "test_matmul", "matmul")
    np_z = z.eval()
    save_idx(np_z, "matmul/output_z.idx")

matmul/w_0.idx saved
matmul/x_0.idx saved
matmul/test_matmul.pb saved
matmul/output_z.idx saved


In [19]:
!python3 quantize_graph.py --input matmul/test_matmul.pb \
  --output matmul/test_quant_matmul.pb --mode=eightbit --output_node_name='z'

In [20]:
np_z

array([[ 2.],
       [ 5.],
       [ 8.]], dtype=float32)

In [21]:
!ls matmul

main.cpp             test_matmul.pb       w_0.idx
output_z.idx         test_quant_matmul.pb x_0.idx


-----

## Test Simple Linear Regression

In [5]:
# testcase for simple linear regression
# the data
X = np.stack([0.5*np.random.randn(50), 0.1*np.random.randn(50)], axis=1)
bias = 0.5
W = np.array([0.29, 0.18])
Y = X.dot(W) + bias + np.random.randn(50)*0.01

In [6]:
X.shape

(50, 2)

In [7]:
Y.shape

(50,)

In [8]:
X

array([[ 0.33910057, -0.02159338],
       [-0.65865954, -0.06241553],
       [ 0.46736623, -0.15260481],
       [-0.22044901,  0.05836734],
       [-0.52837463,  0.02891918],
       [-0.24482941,  0.03069874],
       [ 0.39978895,  0.28506391],
       [-0.0164745 ,  0.067454  ],
       [ 0.22294808,  0.01704514],
       [-0.01217063, -0.02659621],
       [-0.12933585,  0.07948451],
       [-0.19626832,  0.03251618],
       [-0.00434115,  0.00624986],
       [-0.41558219,  0.00935114],
       [-0.20848906,  0.02313185],
       [ 0.03185929,  0.14158923],
       [-0.3518648 , -0.08516396],
       [-0.87177748, -0.03995054],
       [ 0.1644971 , -0.00891856],
       [ 0.65183745,  0.02694414],
       [ 0.009695  , -0.12692682],
       [ 0.74787773, -0.06217783],
       [ 0.72088019, -0.02349306],
       [-0.72923632, -0.01614807],
       [ 0.10889429,  0.03015247],
       [ 0.36309462, -0.00363723],
       [ 0.11783341, -0.06959939],
       [-0.17455983,  0.00783623],
       [ 0.42633517,

In [84]:
graph = tf.Graph()
with graph.as_default():
    tf_x = tf.constant(X, dtype=tf.float32, name="X")
    tf_y = tf.constant(Y, dtype=tf.float32, name="Y")
    
    tf_w = tf.Variable(np.random.randn(2, 1), dtype=tf.float32, name="W")
    tf_b = tf.Variable(0, dtype=tf.float32, name="b")
    tf_yhat = tf.add(tf.matmul(tf_x, tf_w), tf_b, name="yhat")
    loss = tf.reduce_mean(tf.pow(tf_yhat-tf_y, 2), name="loss")
    
    train_op = tf.train.AdamOptimizer(0.001).minimize(loss, name="train_op")

In [85]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    for step in range(1, 10001):
        _, l = sess.run([train_op, loss])
        if step % 1000 == 0:
            print("step:", step)
            print("loss:", l)
            print()
            chkp_path = saver.save(sess, "linreg/chkps/model", global_step=step)
    final_w = tf_w.eval()
    final_b = tf_b.eval()

step: 1000
loss: 0.0169615

step: 2000
loss: 0.0166461

step: 3000
loss: 0.0166461

step: 4000
loss: 0.0166461

step: 5000
loss: 0.0166461

step: 6000
loss: 0.0166461

step: 7000
loss: 0.0166461

step: 8000
loss: 0.0166461

step: 9000
loss: 0.0166461

step: 10000
loss: 0.0166461



In [86]:
final_w, final_b

(array([[  2.84472446e-08],
        [  6.97782525e-08]], dtype=float32), 0.50447685)

In [87]:
from tensorflow.python.framework import graph_util

In [88]:
with tf.Session(graph=graph) as sess:
    saver.restore(sess, chkp_path)
    const_graphdef = graph_util.convert_variables_to_constants(sess, graph.as_graph_def(), ["yhat"])
    with tf.gfile.FastGFile("linreg/test_linreg.pb", "wb") as fid:
        fid.write(const_graphdef.SerializeToString())

INFO:tensorflow:Restoring parameters from linreg/chkps/model-10000


INFO:tensorflow:Restoring parameters from linreg/chkps/model-10000


INFO:tensorflow:Froze 2 variables.


INFO:tensorflow:Froze 2 variables.


Converted 2 variables to const ops.


In [89]:
for n in const_graphdef.node:
    print(n.name, n.op)

X Const
W Const
W/read Identity
b Const
b/read Identity
MatMul MatMul
yhat Add


In [90]:
graph = tf.Graph()
with graph.as_default():
    tf.import_graph_def(const_graphdef, name="")

In [91]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "linreg")
    save_graph(graph, "test_linreg", "linreg")
    pred = graph.get_tensor_by_name("yhat:0")
    np_pred = pred.eval()
    tf_b = graph.get_tensor_by_name("b:0")
    np_b = tf_b.eval()
    save_idx(np_pred, "linreg/output_yhat.idx")

linreg/X_0.idx saved
linreg/W_0.idx saved
linreg/b_0.idx saved
linreg/test_linreg.pb saved
linreg/output_yhat.idx saved


In [92]:
np_b = np.array([np_b]*np_pred.shape[0], dtype=np_b.dtype)

In [93]:
# hacking for uTensor
# uTensor has no broadcasting support
save_idx(np_b, "linreg/b_0.idx")

linreg/b_0.idx saved


In [94]:
!python3 quantize_graph.py --input linreg/test_linreg.pb \
  --output linreg/test_quant_linreg.pb --mode=eightbit --output_node_name='yhat'

In [95]:
!ls linreg

W_0.idx              chkps                output_yhat.idx
X_0.idx              cpp                  test_linreg.pb
b_0.idx              main.cpp             test_quant_linreg.pb


-----

## Test Placeholder

In [34]:
# test case for placeholder
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, name="x")
    one = tf.constant(1.0, dtype=tf.float32, name="one")
    y = tf.add(x, one, name="y")

In [35]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "placeholder")
    save_graph(graph, "test_placeholder", "placeholder")

placeholder/one_0.idx saved
placeholder/test_placeholder.pb saved


In [36]:
!python3 quantize_graph.py --input placeholder/test_placeholder.pb \
  --output placeholder/test_quant_placeholder.pb --mode=eightbit --output_node_name='y'

-----

## Test Relu

In [35]:
# test for relu
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(0.5*np.random.randn(5, 5), name="x", dtype=tf.float32)
    relu = tf.nn.relu(x, name="relu")

In [36]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "relu")
    save_graph(graph, "test_relu", "relu")
    np_relu = relu.eval()
    np_x = x.eval()
    save_idx(np_relu, "relu/output_relu.idx")

relu/x_0.idx saved
relu/test_relu.pb saved
relu/output_relu.idx saved


In [37]:
np_x

array([[-0.80517793, -0.5948562 ,  0.64781803, -0.32989857, -0.61354607],
       [ 0.24788594,  0.09385155, -0.35752174,  0.17100818,  0.06299956],
       [-0.48377144,  0.55246311,  0.01539963,  0.05541649,  0.1231802 ],
       [ 0.13530156, -0.70287359, -0.33819008,  1.02458572,  0.06037853],
       [-0.09076774,  0.56181878,  0.1002373 ,  0.16516355,  0.29926673]], dtype=float32)

In [38]:
!python3 quantize_graph.py --input relu/test_relu.pb \
  --output relu/test_quant_relu.pb --mode=eightbit --output_node_name='relu'

------

## Test Min

In [39]:
!rm -f min_1/*.idx

In [40]:
# test 1 for min op (fixed with hotfix/64)
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.random.rand(2, 2, 2), dtype=tf.float32, name="x")
    min_x = tf.reduce_min(x, name="min_x_1")

In [41]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "min_1")
    save_graph(graph, "test_min_1", "min_1")
    np_min_x = min_x.eval()
    save_idx(np_min_x, "min_1/output_min_x1.idx")

min_1/x_0.idx saved
min_1/Const_0.idx saved
min_1/test_min_1.pb saved
min_1/output_min_x1.idx saved


In [42]:
!python3 quantize_graph.py --input min_1/test_min_1.pb \
  --output min_1/test_quant_min_1.pb --mode=eightbit --output_node_name="min_x_1"

------

In [43]:
!rm -f min_1_1/*.idx

In [44]:
# test 1_1 for min op
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.random.rand(10, 1), dtype=tf.float32, name="x")
    min_x = tf.reduce_min(x, name="min_x_1_1")

In [45]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "min_1_1")
    save_graph(graph, "test_min_1_1", "min_1_1")
    np_min_x = min_x.eval()
    save_idx(np_min_x, "min_1_1/output_min_x1_1.idx")

min_1_1/x_0.idx saved
min_1_1/Const_0.idx saved
min_1_1/test_min_1_1.pb saved
min_1_1/output_min_x1_1.idx saved


In [46]:
!python3 quantize_graph.py --input min_1_1/test_min_1_1.pb \
  --output min_1_1/test_quant_min_1_1.pb --mode=eightbit --output_node_name="min_x_1_1"

-----

In [47]:
!rm -f ./min_2/*.idx

In [48]:
# test 2 for min op
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.random.rand(2, 2, 2), dtype=tf.float32, name="x")
    min_x = tf.reduce_min(x, axis=1, name="min_x_2")

In [49]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "min_2")
    save_graph(graph, "test_min_2", "min_2")
    np_min_x = min_x.eval()
    save_idx(np_min_x, "min_2/output_min_x2.idx")

min_2/x_0.idx saved
min_2/min_x_2_reduction_indices_0.idx saved
min_2/test_min_2.pb saved
min_2/output_min_x2.idx saved


In [50]:
!python3 quantize_graph.py --input min_2/test_min_2.pb \
  --output min_2/test_quant_min_2.pb --mode=eightbit --output_node_name="min_x_2"

-----

In [58]:
!rm -f min_3/*.idx

In [59]:
# test 3 for min op (fail)
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.random.rand(2, 2, 2), dtype=tf.float32, name="x")
    min_x = tf.reduce_min(x, axis=-1, name="min_x_3")  # uTensor do not support neg reduce dim

In [60]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "min_3")
    save_graph(graph, "test_min_3", "min_3")
    np_min_x = min_x.eval()
    save_idx(np_min_x, "min_3/output_min_x3.idx")

min_3/x_0.idx saved
min_3/min_x_3_reduction_indices_0.idx saved
min_3/test_min_3.pb saved
min_3/output_min_x3.idx saved


In [61]:
!python3 quantize_graph.py --input min_3/test_min_3.pb \
  --output min_3/test_quant_min_3.pb --mode=eightbit --output_node_name="min_x_3"

----

In [63]:
# test 3_1 for min op
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.random.rand(2, 2, 2), dtype=tf.float32, name="x")
    min_x = tf.reduce_min(x, axis=2, name="min_x_3_1")

In [64]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "min_3_1")
    save_graph(graph, "test_min_3_1", "min_3_1")
    np_min_x = min_x.eval()
    save_idx(np_min_x, "min_3_1/output_min_x3_1.idx")

min_3_1/x_0.idx saved
min_3_1/min_x_3_1_reduction_indices_0.idx saved
min_3_1/test_min_3_1.pb saved
min_3_1/output_min_x3_1.idx saved


In [65]:
!python3 quantize_graph.py --input min_3_1/test_min_3_1.pb \
  --output min_3_1/test_quant_min_3_1.pb --mode=eightbit --output_node_name="min_x_3_1"

----

## Test Max

In [142]:
!rm -f ./max_1/*.idx

In [143]:
# test 1 for max op (fail! hanging)
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.random.rand(2, 2, 2), dtype=tf.float32, name="x")
    max_x1 = tf.reduce_max(x, name="max_x1")

In [144]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "max_1")
    save_graph(graph, "test_max_1", "max_1")
    np_max_x1 = max_x1.eval()
    save_idx(np_max_x1, "max_1/output_max_x1.idx")

max_1/x_0.idx saved
max_1/Const_0.idx saved
max_1/test_max_1.pb saved
max_1/output_max_x1.idx saved


In [145]:
!python3 quantize_graph.py --input max_1/test_max_1.pb \
  --output max_1/test_quant_max_1.pb --mode=eightbit --output_node_name="max_x1"

-----

In [148]:
!rm -rf ./max_1_1/*.idx

In [150]:
# test 1_1 for max op
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.random.rand(10, 1), dtype=tf.float32, name="x")
    max_x1_1 = tf.reduce_max(x, name="max_x1_1")

In [152]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "max_1_1")
    save_graph(graph, "test_max_1_1", "max_1_1")
    np_max_x1_1 = max_x1_1.eval()
    save_idx(np_max_x1_1, "max_1_1/output_max_x1_1.idx")

max_1_1/x_0.idx saved
max_1_1/Const_0.idx saved
max_1_1/test_max_1_1.pb saved
max_1_1/output_max_x1_1.idx saved


In [153]:
!python3 quantize_graph.py --input max_1_1/test_max_1_1.pb \
  --output max_1_1/test_quant_max_1_1.pb --mode=eightbit --output_node_name="max_x1_1"

----

In [129]:
!rm -rf ./max_2/*.idx

In [130]:
# test 2 for max op
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.random.rand(2, 2, 2), dtype=tf.float32, name="x")
    max_x2 = tf.reduce_max(x, axis=1, name="max_x2")

In [131]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "max_2")
    save_graph(graph, "test_max_2", "max_2")
    np_max_x2 = max_x2.eval()
    save_idx(np_max_x2, "max_2/output_max_x2.idx")

max_2/x_0.idx saved
max_2/max_x2_reduction_indices_0.idx saved
max_2/test_max_2.pb saved
max_2/output_max_x2.idx saved


In [132]:
!python3 quantize_graph.py --input max_2/test_max_2.pb \
  --output max_2/test_quant_max_2.pb --mode=eightbit --output_node_name="max_x2"

-----

In [138]:
!rm -rf ./max_3/*.idx

In [139]:
# test 3 for max op (fail)
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.random.rand(2, 2, 2), dtype=tf.float32, name="x")
    max_x3 = tf.reduce_max(x, axis=-1, name="max_x3") # uTensor do not support neg reduce dim

In [140]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "max_3")
    save_graph(graph, "test_max_3", "max_3")
    np_max_x3 = max_x3.eval()
    save_idx(np_max_x3, "max_3/output_max_x3.idx")

max_3/x_0.idx saved
max_3/max_x3_reduction_indices_0.idx saved
max_3/test_max_3.pb saved
max_3/output_max_x3.idx saved


In [141]:
!python3 quantize_graph.py --input max_3/test_max_3.pb \
  --output max_3/test_quant_max_3.pb --mode=eightbit --output_node_name="max_x3"

-----

In [154]:
!rm -rf ./max_3_1/*.idx

In [155]:
# test 3_1 for max op
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.random.rand(2, 2, 2), dtype=tf.float32, name="x")
    max_x3_1 = tf.reduce_max(x, axis=2, name="max_x3_1") 

In [156]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, "max_3_1")
    save_graph(graph, "test_max_3_1", "max_3_1")
    np_max_x3_1 = max_x3_1.eval()
    save_idx(np_max_x3_1, "max_3_1/output_max_x3_1.idx")

max_3_1/x_0.idx saved
max_3_1/max_x3_1_reduction_indices_0.idx saved
max_3_1/test_max_3_1.pb saved
max_3_1/output_max_x3_1.idx saved


In [158]:
!python3 quantize_graph.py --input max_3_1/test_max_3_1.pb \
  --output max_3_1/test_quant_max_3_1.pb --mode=eightbit --output_node_name="max_x3_1"

-------

## Test Reshape

In [6]:
# test 1 for reshape (fail, since we did not have QuantizedReshape in uTensor)
# flatten
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(np.arange(10).reshape(2, 5),
                    dtype=tf.float32,
                    name='x')
    output_x = tf.reshape(x, shape=[-1], name='x_reshape')

In [7]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, 'reshape_1')
    save_graph(graph, 'test_reshape_1', 'reshape_1')
    np_output = output_x.eval()
    save_idx(np_output, 'reshape_1/output_x.idx')

reshape/x_0.idx saved
reshape/x_reshape_shape_0.idx saved
reshape/test_reshape.pb saved
reshape/output_x.idx saved


In [8]:
!python3 quantize_graph.py --input reshape_1/test_reshape_1.pb \
  --output reshape_1/test_quant_reshape_1.pb --mode=eightbit --output_node_name="x_reshape"

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


-----

In [5]:
# test 2 for reshape (fail, uTensor does not support batch input (Nxk), only 1xk is available)
# the reshape op will be used in this case for tensorflow will flatten 
# the input tensor and find the min/max value for quantized matmul
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32,
                       shape=[None, 3],
                       name='x')
    w = tf.constant(np.random.randn(3, 1),
                    dtype=tf.float32,
                    name='w')
    y = tf.matmul(x, w, name='y')

In [6]:
np_x = np.random.randn(5, 3)

In [7]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, 'reshape_2')
    save_graph(graph, 'test_reshape_2', 'reshape_2')
    np_output = y.eval(feed_dict={'x:0': np_x})
    save_idx(np_x, 'reshape_2/input_x.idx')
    save_idx(np_output, 'reshape_2/output_y.idx')

reshape_2/w_0.idx saved
reshape_2/test_reshape_2.pb saved
reshape_2/input_x.idx saved
reshape_2/output_y.idx saved


In [8]:
np_output.shape

(5, 1)

In [9]:
!python3 quantize_graph.py --input reshape_2/test_reshape_2.pb \
  --output reshape_2/test_quant_reshape_2.pb --mode=eightbit --output_node_name="y"

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


-----

In [12]:
# test 3 for reshape (fail....why??)
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32,
                       shape=[1, 5],
                       name='x')
    w = tf.constant(np.random.randn(5, 3),
                    dtype=tf.float32,
                    name='w')
    y = tf.matmul(x, w, name='y')

In [13]:
np_x = np.random.randn(1, 5)

In [14]:
with tf.Session(graph=graph) as sess:
    save_consts(sess, 'reshape_3')
    save_graph(graph, 'test_reshape_3', 'reshape_3')
    np_output = y.eval(feed_dict={'x:0': np_x})
    save_idx(np_x, 'reshape_3/input_x.idx')
    save_idx(np_output, 'reshape_3/output_y.idx')

reshape_3/w_0.idx saved
reshape_3/test_reshape_3.pb saved
reshape_3/input_x.idx saved
reshape_3/output_y.idx saved


In [15]:
!python3 quantize_graph.py --input reshape_3/test_reshape_3.pb \
  --output reshape_3/test_quant_reshape_3.pb --mode=eightbit --output_node_name="y"

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


-----

In [29]:
!make clean

removing add/cpp
removing argmax/cpp
removing const/cpp
removing matmul/cpp
removing linreg/cpp
removing placeholder/cpp
removing relu/cpp
removing min_1/cpp
removing min_1_1/cpp
removing min_2/cpp
removing min_3/cpp
removing min_3_1/cpp
removing max_1_1/cpp
removing max_1/cpp
removing max_2/cpp
removing max_3/cpp
removing max_3_1/cpp
removing reshape_1/cpp
removing reshape_2/cpp
rm -rf linreg/chkps
